<a href="https://colab.research.google.com/github/RJAbuNasser/Final-Project/blob/main/Final_Project_Girls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Improting Libraries**

In [2]:
import zipfile
import os
import pandas as pd
import io

**Access Files & Data**

In [3]:
def enter_txt(file):
    lines = file.readlines()
    data = [line.decode('utf-8').strip().replace(',', ' ').split() for line in lines]
    results = pd.DataFrame(data)
    print(results)

def enter_xlsx(file):
    df = pd.read_excel(io.BytesIO(file.read()), sheet_name=None)
    for sheet_name, sheet_data in df.items():
        print(f"Sheet: {sheet_name}")
        print(sheet_data)

def enter_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for zip_file in zip_ref.namelist():
            if 'Results' in zip_file or '-D' in zip_file:
                folder_name = zip_file.split('/')[0]
                if folder_name.startswith('CEC'):
                    print(f"Entering CEC folder: {folder_name}")
                elif folder_name.startswith('com'):
                    print(f"Entering com folder: {folder_name}")
                elif 'GECCO1' in folder_name:
                    print(f"Entering GECCO1 folder: {folder_name}")
                elif folder_name.startswith('report'):
                    print(f"Entering report folder: {folder_name}")
                if zip_file.endswith('.txt'):
                    print(f"Entering txt file: {zip_file}")
                    with zip_ref.open(zip_file) as file:
                        enter_txt(file)
                elif zip_file.endswith('.xlsx'):
                    print(f"Entering xlsx file: {zip_file}")
                    with zip_ref.open(zip_file) as file:
                        enter_xlsx(file)
enter_zip("Data-All Algos.zip")

Streaming output truncated to the last 5000 lines.
14  52761.03888287999  444125506.2055  
15  52761.03888287999  444125506.2055  
16              50786            1724  

[17 rows x 30 columns]
Entering txt file: Data-All Algos/report1/Results/SPHH_Ensemble_7_10.txt
                       0                      1                   2   \
0      242.57826210399998          275.195985437        320.87631613   
1      118.44356260199997     117.59531061999999  329.19882848500004   
2            76.394710011          130.162066581        186.09762444   
3           76.5724213697     104.69166577979999       171.780368557   
4            81.775439813           40.226752102       213.026630023   
5           157.289890154          194.393011144  136.52073919900002   
6      139.33578034499996           133.26506457       117.707422455   
7           364.514269625          111.959822893        78.581736956   
8    0.008600044000000001     261.56822057000005         0.018093611   
9    0.00860